In [ ]:
# Parameters: 
affiliation = 'AfD'
aggregation = 'week' # aggregation can be by day ('day'), week ('week') or month('month')
from_date='2017-09-01'
to_date='2018-12-31'

## using API

In [ ]:
import pandas as pd
from smm_wrapper import SMMPoliticians

# Create an instance to the smm wrapper
smm = SMMPoliticians()

#using the api to get the data
df = smm.dv.get_politicians()

# Filter the accounts by party, and valid ones (the ones that contain tw_ids)
party_df = df[(df['affiliation']==affiliation) & (df['tw_ids'].notnull())] 

# query the Social Media Monitoring API
tweets_by = pd.concat(smm.dv.tweets_by(twitter_user_id=tw_id, from_date=from_date, to_date=to_date, aggregate_by=aggregation) 
               for tw_ids in party_df['tw_ids'] for tw_id in tw_ids)        
replies_to = pd.concat(smm.dv.replies_to(twitter_user_id=tw_id, from_date=from_date, to_date=to_date, aggregate_by=aggregation) 
               for tw_ids in party_df['tw_ids'] for tw_id in tw_ids) 

# aggregate the replies
total_tweets_by = tweets_by.groupby('date').agg({'tweets': 'sum'})
total_replies_to = replies_to.groupby('date').agg({'replies': 'sum'})

## Plotting

In [ ]:
#plotting data
import plotly
from plotly import graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

plotly.offline.iplot({
    "data": [go.Scatter(x=total_tweets_by.index.tolist(), y=total_tweets_by['tweets'], name='Tweets', line_shape='spline'), 
             go.Scatter(x=total_replies_to.index.tolist(), y=total_replies_to['replies'], name='Replies', line_shape='spline')], 
    "layout": go.Layout( title='Tweets and replies for {} party'.format(affiliation), xaxis={'title':''}, yaxis={'title':'N'})
})